# Clustering Crypto

In [1]:
!pip install -U altair


    100% |████████████████████████████████| 737kB 23.1MB/s ta 0:00:01
Requirement not upgraded as not directly required: entrypoints in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.2.3)
Requirement not upgraded as not directly required: jsonschema in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (2.6.0)
Requirement not upgraded as not directly required: jinja2 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (2.10)
Requirement not upgraded as not directly required: toolz in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.9.0)
Requirement not upgraded as not directly required: numpy in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (1.14.3)
Requirement not upgraded as not directly required: pandas>=0.18 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.24.2)
Requirement not upgrad

In [19]:
# Initial imports
import altair as alt
import pandas as pd
#import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Data Preprocessing

In [5]:
# Loade the cryptocurrencies data
file_path = Path("crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [6]:
# Keep only cryptocurrencies that are on trading
crypto_df=crypto_df.loc[crypto_df['IsTrading']==True]

In [7]:
# Remove the "IsTrading" column
crypto_df.drop(columns='IsTrading',inplace=True)

In [8]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [9]:
# Remove rows with cryptocurrencies without coins mined
crypto_df=crypto_df.loc[crypto_df['TotalCoinsMined']!=0]


In [10]:
# Fetch the cryptocurrencies names prior to drop them from crypto_df
coin_names=pd.DataFrame({'CoinName':crypto_df['CoinName']})
coin_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [11]:
# Remove the cryptocurrency name since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName',inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [12]:
# Create dummies variables for text features
X=pd.get_dummies(crypto_df,columns=['Algorithm','ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Standardize data
X= StandardScaler().fit_transform(X)
X

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype uint8, float64, object were all converted to float64 by StandardScaler.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype uint8, float64, object were all converted to float64 by StandardScaler.



array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [14]:
# Use PCA to reduce dimension to 3 principal components
pca = PCA(n_components=3)

# Get two principal components for the iris data.
crypto_pca = pca.fit_transform(X)

In [15]:
# Create a DataFrame with the principal components data
crypto_df_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2",'PC 3'],index=crypto_df.index
)
crypto_df_pca.head()


,PC 1,PC 2,PC 3
42,-0.322017,0.983992,-0.319018
404,-0.305368,0.984194,-0.319233
1337,2.277672,1.579952,-0.409843
BTC,-0.157752,-1.332895,0.064550
ETH,-0.138656,-2.013309,0.236314


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [22]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df_pca)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(
    x="k", y="inertia",
)

alt.Chart(...)

Running K-Means with `k=4`

In [23]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_df_pca)
# Predict clusters
predictions = model.predict(crypto_df_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features

clustered_df=pd.concat([crypto_df,crypto_df_pca,coin_names], axis=1)
clustered_df["Class"] = model.labels_
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.322017,0.983992,-0.319018,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.305368,0.984194,-0.319233,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.277672,1.579952,-0.409843,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.157752,-1.332895,0.064550,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.138656,-2.013309,0.236314,Ethereum,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [24]:
# Create a 3D-Scatter with the PCA data and the clusters

alt.Chart(clustered_df).mark_circle(size=60).encode(
   x='PC 1',
    y='PC 2',
    color="Class",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
# Table with tradable cryptos
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        clustered_df[
            [
                "CoinName",
                "Algorithm",
                "ProofType",
                "TotalCoinSupply",
                "TotalCoinsMined",
                "Class",
            ]
        ]
    )

In [57]:
# Print the total number of tradable cryptocurrencies
total_number=clustered_df.shape[0]
print(f'The total number of tradable cryptocurrencies is: {total_number}')

The total number of tradable cryptocurrencies is: 533


#### Scatter Plot with Tradable Cryptocurrencies

In [27]:
# Scale data to create the scatter plot
scaled_clus=StandardScaler().fit_transform(clustered_df.iloc[:,2:4])
scaled_clus=pd.DataFrame({"TotalCoinsMined":scaled_clus[:,0],"TotalCoinSupply":scaled_clus[:,1],'CoinName':coin_names.iloc[:,0]})
scaled_clus['Class']=model.labels_
scaled_clus


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype float64, object were all converted to float64 by StandardScaler.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype float64, object were all converted to float64 by StandardScaler.



,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,-0.116748,-0.152865,42 Coin,3
404,-0.093589,-0.144996,404Coin,3
1337,0.525872,4.493764,EliteCoin,3
BTC,-0.116354,-0.152554,Bitcoin,0
ETH,-0.114384,-0.152865,Ethereum,0
LTC,-0.115364,-0.151622,Litecoin,0
DASH,-0.116550,-0.152539,Dash,3
XMR,-0.116370,-0.152865,Monero,0
ETC,-0.114260,-0.149759,Ethereum Classic,0
ZEC,-0.116586,-0.152554,ZCash,0


In [28]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(scaled_clus).mark_circle(size=60).encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    color="Class",
).interactive()

alt.Chart(...)